# Imports

In [1]:
from google.cloud import storage
import pandas as pd
import numpy as np
import re

/tmp/ipykernel_1514370/4223526492.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Carregando Dados

In [2]:
BUCKET_NAME = 'busca-apartamentos-bucket'
FILE = '2024-02-14 - apartamentos - apolar.csv'
TEMP_FILE = 'local.csv'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

blop = bucket.blob(blob_name= FILE).download_as_string()
with open(TEMP_FILE, 'wb') as f:
    f.write(blop)

df = pd.read_csv('local.csv')

# Tratando Dados

## Limpeza

In [3]:
df['titulo'] = df['titulo'].str.strip()
df['endereco'] = df['endereco'].str.strip()
df['descricao'] = df['descricao'].str.strip().str.replace('  ', '').str.replace('\n',' ')

# Criação de Features

In [4]:
def busca_substring(substring, string_list):
    result = np.nan
    for s in string_list:
        if substring in s:
            try:
                result = re.findall(r'\s(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)', s)[0]
            except:
                result = s
            break
            
    return result

def separa_valores_imovel(string):

    # Padrao regex para encontrar nome e valor monetário
    padrao = r'(\w+)\sR\$\s(\d{1,3}(?:\.\d{3})*(?:,\d{2})?)'

    # Encontrar todas as correspondências na string
    correspondencias = re.findall(padrao, string)

    # Imprimir os resultados
    list_values = []
    for correspondencia in correspondencias:
        nome, valor = correspondencia
        list_values.append(f'{nome}: {valor}')
    
    return list_values

In [5]:
# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x.split(', ')[2].split(' - ')[0])
df['cidade'] = df['endereco'].apply(lambda x: x.split(', ')[2].split(' -')[1])

# Atributos do imóvel
df['area'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('m²', x.split(', ')))
df['banheiros'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('banheiro', x.split(', ')))
df['vagas_garagem'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('vagas', x.split(', ')))
df['quartos'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('quarto', x.split(', ')))

df['area'] = df['area'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['banheiros'] = df['banheiros'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['vagas_garagem'] = df['vagas_garagem'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['quartos'] = df['quartos'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')

# Valores
df['aluguel'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Aluguel',separa_valores_imovel(x)))
df['condominio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Condomínio',separa_valores_imovel(x)))
df['seguro_incendio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Incêndio',separa_valores_imovel(x)))
df['iptu'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('IPTU',separa_valores_imovel(x)))

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['seguro_incendio'] = df['seguro_incendio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')

# Detalhes do imóvel/condomínio
df['mobiliado'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in x else 'Não')
df['piscina'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in x else 'Não')
df['academia'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in x else 'Não')
df['sacada'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in x else 'Não')
df['churrasqueira'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in x else 'Não')

In [6]:
df.dtypes

site                object
data_coleta         object
titulo              object
link                object
endereco            object
valores             object
atributos           object
descricao           object
bairro              object
cidade              object
area               float64
banheiros          float64
vagas_garagem      float64
quartos            float64
aluguel            float64
condominio         float64
seguro_incendio    float64
iptu               float64
mobiliado           object
piscina             object
academia            object
sacada              object
churrasqueira       object
dtype: object

# Save File

In [7]:
columns_selected = [
 'site',
 'link',
 'data_coleta',
 'titulo',
 'endereco',
 'atributos',
 'descricao',
 'bairro',
 'cidade',
 'area',
 'banheiros',
 'vagas_garagem',
 'quartos',
 'aluguel',
 'condominio',
 'seguro_incendio',
 'iptu',
 'mobiliado',
 'piscina',
 'academia',
 'sacada',
 'churrasqueira'
 ]

In [8]:
df.columns

Index(['site', 'data_coleta', 'titulo', 'link', 'endereco', 'valores',
       'atributos', 'descricao', 'bairro', 'cidade', 'area', 'banheiros',
       'vagas_garagem', 'quartos', 'aluguel', 'condominio', 'seguro_incendio',
       'iptu', 'mobiliado', 'piscina', 'academia', 'sacada', 'churrasqueira'],
      dtype='object')

In [9]:
df[columns_selected].to_excel('../data/data_trusted/anuncios-apolar-trusted.xlsx', index = False)